#### Task 2
### Preprocessing
- [x] Удалить HTML (re.sub(pattern, substract_with, text), replace)
- [x] Токенизация (nltk.tokenize.word_tokenize)
- [x] Удалить стоп-слов и знаки препинания (перебор порога)
- - самые часто встречаемые и редкие слова
- - Pointwise Mutual Information
- - Inverse Document Frequency
- [x] Приведение к нижнему регистру
- [x] Лемматизация или Стемминг

*Davletyarov Ildar, 11-808*

In [9]:
import nltk

# stab for download nltk packages without SSL
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/ildar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from collections import defaultdict
import string
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.probability import FreqDist
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package wordnet to /Users/ildar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ildar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# load data
DATA_PATH = "/Users/ildar/Desktop/labeledTrainData.tsv"
df = pd.read_csv(DATA_PATH, sep='\t', encoding='utf-8')

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [12]:
df.to_dict('records')[0]

{'id': '5814_8',
 'sentiment': 1,
 'review': "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />Th

In [13]:
COUNT_WORDS = 500

texts = df.review.tolist()[:COUNT_WORDS]
sentiment = df.sentiment.tolist()[:COUNT_WORDS]
for i in range(len(sentiment)):
    sentiment[i] = float(sentiment[i])

In [23]:
def delete_html(text):
    return text.replace('<br />', '')

def transform_text(text, spec_chars, lemmatizer, frequency):
    w = []
    text = text.lower()
    word_list = nltk.word_tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    text = "".join([ch for ch in text if ch not in spec_chars])
    text = delete_html(text)
    fdist = FreqDist(text)
    freq = fdist.most_common(15)
    freq
    text
    for j in range(15):
        if freq[j][1] >= 4:
            w.append(freq[j][0])
    if len(w) > 0:
        for k in range(len(w)):
            text = text.replace(w[k], '')
    for w in text:
        if freq[w] < 0.143:
            text = text.replace(w, "")
    stop_words = set(stopwords.words("english"))
    words = nltk.word_tokenize(text)
    without_stop_words = [word for word in words if not word in stop_words]
    return without_stop_words

In [15]:
class NaiveBayesClassifier:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.class_probs = {}
        self.conditional_probabilities = {}
        self._class_freq = defaultdict(lambda: 0)
        self._feat_freq = defaultdict(lambda: 0)
    
    def fit(self, X, y):
        for feature, label in zip(X, y):
            self._class_freq[label] += 1
            for value in feature:
                self._feat_freq[(value, label)] += 1
        
        num_samples = len(X)
        for k in self._class_freq:
            self._class_freq[k] /= num_samples
            
        for value, label in self._feat_freq:
            self._feat_freq[(value, label)] /= self._class_freq[label]
            
        return self       
        
    def predict(self, X):
        return min(self._class_freq.keys(), key=lambda c : self._calculate_class_freq(X, c))
    
    def _calculate_class_freq(self, X, clss):
        freq = -np.log(self._class_freq[clss])
        for feat in X:
            freq += -np.log(self._feat_freq.get((feat, clss), 10 ** (-7)))
        return freq

In [26]:
import math
frequency = {}
for text in texts: # count of words
    for word in text:
        count = frequency.get(word,0)
        frequency[word] = count + 1

for word in frequency.keys(): # idf
    osn = COUNT_WORDS / frequency[word]
    frequency[word] = math.log(osn, 2)

In [27]:
spec_chars = string.punctuation + '\n\t'
lemmatizer = WordNetLemmatizer()

for i in range(len(texts)):
    texts[i] = transform_text(texts[i], spec_chars, lemmatizer, frequency)

TypeError: list indices must be integers or slices, not str

In [28]:
freq_pmi = {} # pmi
for i in range(len(texts)):
    for j in range(len(texts[i])):
        count = freq_pmi.get(zip(texts[i][j], str(sentiment[i])),0)
        freq_pmi[zip(texts[i][j], str(sentiment[i]))] = count + 1
          
for w in freq_pmi.keys():
    bim = 500 / freq_pmi[w]
    freq_pmi[w] = math.log(bim / (osn * 0.5), 2)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(texts, sentiment)

In [30]:
model = NaiveBayesClassifier(n_classes=2)
model.fit(X_train, y_train)
y_pred = [model.predict(x) for x in X_test]

In [40]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.78      0.82      0.80      3033
           1       0.82      0.78      0.80      3217

    accuracy                           0.80      6250
   macro avg       0.80      0.80      0.80      6250
weighted avg       0.80      0.80      0.80      6250



In [31]:
len(y_pred)

125

In [ ]:
# repeat with text preprocessing